In [1]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LightSource


%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [17]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,Dataset
import pandas as pd

from torch.optim.optimizer import Optimizer
import numpy as np
import random
dataset_="CIFAR10"
# import autograd_hacks
# from collections import defaultdict
# import sys
# from autograd_lib import autograd_lib
# from autograd_lib import util as
test_loader=[]
batch_size_ = 128
train_loader=[]
if dataset_=="CIFAR10":
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                              shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=128,
                                             shuffle=False)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # inputs, labels = data[0].to(device), data[1].to(device)
else:
    print("test2")
    mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
    mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())
    train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
    test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



Files already downloaded and verified
Files already downloaded and verified


In [18]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
torch.manual_seed(0)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)
model_cnn=[]

if dataset_ == "CIFAR10" :
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 6, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(6, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)
        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 16 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x
    model_cnn = Net()
    model_cnn.to(device)
else:
   model_cnn = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.ReLU(),
                          nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(64, 64, 3, padding=1, stride=2), nn.ReLU(),
                          Flatten(),
                          nn.Linear(7*7*64, 100), nn.ReLU(),
                          nn.Linear(100, 10)).to(device)

def epoch(loader, model, opt=None):

    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

In [20]:
optims= [optim.SGD(model_cnn.parameters(), lr=0.01),optim.SGD(model_cnn.parameters(), lr=0.01,momentum=0.9),optim.Adam(model_cnn.parameters(), lr=0.001, betas=(0.9, 0.999))]
opts_=["SGD","SGD with momentum =0.9","ADAM lr=0.001, betas=(0.9, 0.999)"]
for opt,opt_ in zip(optims,opts_):
    for i in range(6):
        train_err=[]
        test_err=[]

        for t in range(10):
            train_err, train_loss = epoch(train_loader, model_cnn, opt)
            test_err, test_loss = epoch(test_loader, model_cnn)
            if opt_ =="SGD":
                if t == 4:
                    for param_group in opt.param_groups:
                        param_group["lr"] = 1e-2
            print(*("{:.6f}".format(i) for i in (train_err, test_err, )), sep="\t")

        column_names = ["OPT", "Train", "Test"]
        dic = {dataset_:[opt_,train_err,test_err]}

        df = pd.DataFrame.from_dict(dic,orient='index',columns=column_names)

        df.to_csv('Experiment_results.csv', mode='a', header=False)

        model_cnn.apply(weight_reset)

KeyboardInterrupt: 